# Preliminary Modeling

In [33]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

from raw_to_transformed_data import get_sql_data

## Importing data

In [34]:
query_crashes = """
    SELECT *
    FROM crashes;
    """
dbname = "chi-traffic-accidents"

In [35]:
df_crashes = get_sql_data(dbname, query_crashes)

In [36]:
drop_cols = ['crash_record_id', 'crash_date', 'report_type', 'prim_contributory_cause', 'intersection_related_i', 'hit_and_run_i', 'lane_cnt', 'has_injuries']
df_crashes = df_crashes.drop(columns=drop_cols)

df_crashes = df_crashes.rename(columns={"crash_day_of_week": "crash_day"})
df_crashes["street_direction"] = (
    df_crashes["street_direction"]
        .fillna(df_crashes["street_direction"].mode()[0]))

In [37]:
df_crashes.head()

,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,street_direction,num_units,injuries_total,crash_hour,crash_day,crash_month
0,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,S,2,0.0,3,Sunday,August
1,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,N,2,1.0,2,Sunday,August
2,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,S,2,0.0,2,Sunday,August
3,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,S,2,1.0,2,Sunday,August
4,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,W,3,2.0,2,Sunday,August


In [38]:
y = df_crashes.pop("injuries_total")

In [39]:
y[:5]

0    0.0
1    1.0
2    0.0
3    1.0
4    2.0
Name: injuries_total, dtype: float64

In [40]:
X = df_crashes.copy()

In [41]:
X[:5]

,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,street_direction,num_units,crash_hour,crash_day,crash_month
0,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,S,2,3,Sunday,August
1,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,N,2,2,Sunday,August
2,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,S,2,2,Sunday,August
3,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,S,2,2,Sunday,August
4,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,W,3,2,Sunday,August


# Transforming data

In [42]:
numeric_cols = ["posted_speed_limit", "num_units", "crash_hour"]

In [43]:
category_cols = df_crashes.columns.difference(numeric_cols)

In [44]:
category_cols

Index(['alignment', 'crash_day', 'crash_month', 'device_condition',
       'first_crash_type', 'lighting_condition', 'road_defect',
       'roadway_surface_cond', 'street_direction', 'traffic_control_device',
       'trafficway_type', 'weather_condition'],
      dtype='object')

In [45]:
encoder=OneHotEncoder(drop=None, sparse=True)

In [46]:
onehot_crashes = encoder.fit_transform(X[category_cols])

In [47]:
matrix_cols = []
for col, ele in zip(category_cols, encoder.categories_):
    for e in ele:
        matrix_cols.append(col + "_" + e.lower())

In [48]:
X = pd.concat([X[numeric_cols], pd.DataFrame(onehot_crashes.toarray(), columns=matrix_cols)], axis=1)

In [49]:
X

,posted_speed_limit,num_units,crash_hour,alignment_curve on grade,alignment_curve on hillcrest,"alignment_curve, level",alignment_straight and level,alignment_straight on grade,alignment_straight on hillcrest,crash_day_friday,crash_day_monday,crash_day_saturday,crash_day_sunday,crash_day_thursday,crash_day_tuesday,crash_day_wednesday,crash_month_april,crash_month_august,crash_month_december,crash_month_february,crash_month_january,crash_month_july,crash_month_june,crash_month_march,crash_month_may,crash_month_november,crash_month_october,crash_month_september,device_condition_functioning improperly,device_condition_functioning properly,device_condition_missing,device_condition_no controls,device_condition_not functioning,device_condition_other,device_condition_unknown,device_condition_worn reflective material,first_crash_type_angle,first_crash_type_animal,first_crash_type_fixed object,first_crash_type_head on,first_crash_type_other noncollision,first_crash_type_other object,first_crash_type_overturned,first_crash_type_parked motor vehicle,first_crash_type_pedalcyclist,first_crash_type_pedestrian,first_crash_type_rear end,first_crash_type_rear to front,first_crash_type_rear to rear,first_crash_type_rear to side,first_crash_type_sideswipe opposite direction,first_crash_type_sideswipe same direction,first_crash_type_train,first_crash_type_turning,lighting_condition_darkness,"lighting_condition_darkness, lighted road",lighting_condition_dawn,lighting_condition_daylight,lighting_condition_dusk,lighting_condition_unknown,road_defect_debris on roadway,road_defect_no defects,road_defect_other,"road_defect_rut, holes",road_defect_shoulder defect,road_defect_unknown,road_defect_worn surface,roadway_surface_cond_dry,roadway_surface_cond_ice,roadway_surface_cond_other,"roadway_surface_cond_sand, mud, dirt",roadway_surface_cond_snow or slush,roadway_surface_cond_unknown,roadway_surface_cond_wet,street_direction_e,street_direction_n,street_direction_s,street_direction_w,traffic_control_device_bicycle crossing sign,traffic_control_device_delineators,traffic_control_device_flashing control signal,traffic_control_device_lane use marking,traffic_control_device_no controls,traffic_control_device_no passing,traffic_control_device_other,traffic_control_device_other railroad crossing,traffic_control_device_other reg. sign,traffic_control_device_other warning sign,traffic_control_device_pedestrian crossing sign,traffic_control_device_police/flagman,traffic_control_device_railroad crossing gate,traffic_control_device_rr crossing sign,traffic_control_device_school zone,traffic_control_device_stop sign/flasher,traffic_control_device_traffic signal,traffic_control_device_unknown,traffic_control_device_yield,trafficway_type_alley,trafficway_type_center turn lane,trafficway_type_divided - w/median (not raised),trafficway_type_divided - w/median barrier,trafficway_type_driveway,"trafficway_type_five point, or more",trafficway_type_four way,trafficway_type_l-intersection,trafficway_type_not divided,trafficway_type_not reported,trafficway_type_one-way,trafficway_type_other,trafficway_type_parking lot,trafficway_type_ramp,trafficway_type_roundabout,trafficway_type_t-intersection,trafficway_type_traffic route,trafficway_type_unknown,trafficway_type_unknown intersection type,trafficway_type_y-intersection,"weather_condition_blowing sand, soil, dirt",weather_condition_blowing snow,weather_condition_clear,weather_condition_cloudy/overcast,weather_condition_fog/smoke/haze,weather_condition_freezing rain/drizzle,weather_condition_other,weather_condition_rain,weather_condition_severe cross wind gate,weather_condition_sleet/hail,weather_condition_snow,weather_condition_unknown
0,30,2,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

## Baseline Model

In [50]:
X.shape

(526569, 129)

In [51]:
y.shape

(526569,)

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [53]:
model_dum = DummyRegressor(strategy="mean")
model_dum.fit(X_train, y_train)

DummyRegressor()

In [54]:
y_pred = model_dum.predict(X_test)

In [55]:
y_pred[:5]

array([0.18061865, 0.18061865, 0.18061865, 0.18061865, 0.18061865])

In [56]:
y_test[:5]

342410    0.0
262477    0.0
227422    0.0
514106    0.0
105362    0.0
Name: injuries_total, dtype: float64

In [57]:
np.sqrt(mean_squared_error(y_test, y_pred))

0.5491703132053445

## Linear Regression

In [58]:
import time

In [59]:
# X_train, X_test, y_train, y_test = train_test_split(df_crashes, y)

In [60]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

LinearRegression()

In [61]:
y_pred_lr = model_lr.predict(X_test)

In [62]:
np.sqrt(mean_squared_error(y_test, y_pred_lr))

0.5142709692712032

## Random Forest

In [63]:
n_est_rf = 100

In [64]:
model_rf = RandomForestRegressor(
    n_estimators=n_est_rf,
    max_features="auto")

start_time = time.time()
model_rf.fit(X_train, y_train)
print(time.time()-start_time)

766.1581768989563


In [65]:
start_time = time.time()
y_pred_rf = model_rf.predict(X_test)
print(time.time()-start_time)

5.021013975143433


In [66]:
np.sqrt(mean_squared_error(y_test, y_pred_rf))

0.5477801568325902

## GradientBoost

In [69]:
model_gb = GradientBoostingRegressor(
    learning_rate=0.1,
    n_estimators=100,
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2)

start_time = time.time()
model_gb.fit(X_train, y_train)
print(time.time()-start_time)

118.20394802093506


In [70]:
start_time = time.time()
y_pred_gb = model_gb.predict(X_test)
print(time.time()-start_time)

0.7527539730072021


In [71]:
np.sqrt(mean_squared_error(y_test, y_pred_gb))

0.5105910306942633

In [67]:
df_crashes.head()

,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,street_direction,num_units,crash_hour,crash_day,crash_month
0,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,S,2,3,Sunday,August
1,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,N,2,2,Sunday,August
2,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,S,2,2,Sunday,August
3,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,S,2,2,Sunday,August
4,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,W,3,2,Sunday,August
